In [ ]:
#BM Talos Single headed network year 1 hyperparameter search w/results
#full hyperparameter seaarch talos DNN/single headed(SH) model

#months 1-7 search

import numpy as np
import pandas as pd
from pandas import concat
import matplotlib.pyplot as plt
from functools import reduce
import importlib
import datetime as dt
from datetime import datetime
from math import floor
import seaborn as sns
from math import sqrt


#tensorflow &keras
import tensorflow as tf
from keras.layers import Input
from keras.models import Model
from tensorflow.keras.layers import GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras import optimizers
from tensorflow.keras import initializers
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.layers import concatenate
from keras.layers import Flatten
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
# from keras.wrappers.scikit_learn import KerasClassifier

# from keras.layers.Wrappers.scikit_learn import KerasRegressor
from scikeras.wrappers import KerasRegressor

from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
from keras.activations import *

from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)

#sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn import metrics
from sklearn import preprocessing as prep
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score, make_scorer
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, mean_absolute_error
from sklearn.metrics import mean_squared_error as MSE
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import talos 
import tensorflow.keras.backend as K

import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

date_format = "%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("C:/Users/coconnor/Documents/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_parser=date_parse)
# dat = pd.read_csv("./data/BM_data.csv", index_col="SettlementPeriod", parse_dates=True,
#                   date_parser=date_parse)
dat = pd.read_csv("/home/ciaran/Documents/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_parser=date_parse)
#dat = pd.read_csv("/home/coconnor/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_parser=date_parse)

dat = dat.drop(["index"], axis=1)
dat = pd.DataFrame(dat)
dat = dat.bfill(axis='rows')
dat = dat.ffill(axis='rows')
dat = dat._get_numeric_data()

Y=dat.iloc[:, 0:16]
X=dat.iloc[:,16:]
X_train=X.iloc[:7250,:]
Y_train=Y.iloc[:7250,:]
X_test=X.iloc[7250:8739,:]
Y_test=Y.iloc[7250:8739,:]

def qloss(qs, y_true, y_pred):
    # Pinball loss for multiple quantiles
    q = tf.constant(np.array([qs]), dtype=tf.float32)
    e = y_true - y_pred
    v = tf.maximum(q*e, (q-1)*e)
    return K.mean(v)

loss_10th_p = lambda y_true, y_pred: qloss(0.1, y_true, y_pred)
loss_30th_p = lambda y_true, y_pred: qloss(0.3, y_true, y_pred)
loss_50th_p = lambda y_true, y_pred: qloss(0.5, y_true, y_pred)
loss_70th_p = lambda y_true, y_pred: qloss(0.7, y_true, y_pred)
loss_90th_p = lambda y_true, y_pred: qloss(0.9, y_true, y_pred)

rnn_train_LSTM_1 = X_train.loc[:,"lag_-3x1":"lag_-50x1"]
rnn_train_LSTM_2 = X_train.loc[:,"lag_-3x2": "lag_-50x2"]
rnn_train_LSTM_3 = X_train.loc[:,"lag_-2x3":"lag_-49x3"]
rnn_train_LSTM_4 = X_train.loc[:,"lag_0x6":"lag_-47x6"]
rnn_train_LSTM_5 = X_train.loc[:,"lag_-2x12": "lag_-49x12"]

rnn_test_LSTM_1 = X_test.loc[:,"lag_-3x1": "lag_-50x1"]
rnn_test_LSTM_2 = X_test.loc[:,"lag_-3x2": "lag_-50x2"]
rnn_test_LSTM_3 = X_test.loc[:,"lag_-2x3": "lag_-49x3"]
rnn_test_LSTM_4 = X_test.loc[:,"lag_0x6" : "lag_-47x6"]
rnn_test_LSTM_5 = X_test.loc[:,"lag_-2x12": "lag_-49x12"]

rnn_train_ffnn_1 = X_train.loc[:,"lag_2x7": "lag_17x7"]
rnn_train_ffnn_2 = X_train.loc[:,"lag_2x8": "lag_17x8"]
rnn_train_ffnn_3 = X_train.loc[:,"lag_2x9": "lag_17x9"]
rnn_train_ffnn_4 = X_train.loc[:,"lag_2x10": "lag_17x10"]
rnn_train_ffnn_5 = X_train.loc[:,"lag_2x11": "lag_17x11"]

rnn_test_ffnn_1 = X_test.loc[:,"lag_2x7":"lag_17x7"]
rnn_test_ffnn_2 = X_test.loc[:,"lag_2x8": "lag_17x8"]
rnn_test_ffnn_3 = X_test.loc[:,"lag_2x9": "lag_17x9"]
rnn_test_ffnn_4 = X_test.loc[:,"lag_2x10":"lag_17x10"]
rnn_test_ffnn_5 = X_test.loc[:,"lag_2x11": "lag_17x11"]

rnn_Y = Y_train.loc[:,"lag_2y": "lag_17y"]

X_scaler_LSTM_1 = preprocessing.MinMaxScaler()
X_scaler_LSTM_2 = preprocessing.MinMaxScaler()
X_scaler_LSTM_3 = preprocessing.MinMaxScaler()
X_scaler_LSTM_4 = preprocessing.MinMaxScaler()
X_scaler_LSTM_5 = preprocessing.MinMaxScaler()

X_scaler_ffnn_1 = preprocessing.MinMaxScaler()
X_scaler_ffnn_2 = preprocessing.MinMaxScaler()
X_scaler_ffnn_3 = preprocessing.MinMaxScaler()
X_scaler_ffnn_4 = preprocessing.MinMaxScaler()
X_scaler_ffnn_5 = preprocessing.MinMaxScaler()

Y_scaler = preprocessing.MinMaxScaler()

rnn_scaled_train_LSTM_1 = X_scaler_LSTM_1.fit_transform(rnn_train_LSTM_1)
rnn_scaled_train_LSTM_2 = X_scaler_LSTM_2.fit_transform(rnn_train_LSTM_2)
rnn_scaled_train_LSTM_3 = X_scaler_LSTM_3.fit_transform(rnn_train_LSTM_3)
rnn_scaled_train_LSTM_4 = X_scaler_LSTM_4.fit_transform(rnn_train_LSTM_4)
rnn_scaled_train_LSTM_5 = X_scaler_LSTM_5.fit_transform(rnn_train_LSTM_5)

rnn_scaled_train_ffnn_1 = X_scaler_ffnn_1.fit_transform(rnn_train_ffnn_1)
rnn_scaled_train_ffnn_2 = X_scaler_ffnn_2.fit_transform(rnn_train_ffnn_2)
rnn_scaled_train_ffnn_3 = X_scaler_ffnn_3.fit_transform(rnn_train_ffnn_3)
rnn_scaled_train_ffnn_4 = X_scaler_ffnn_4.fit_transform(rnn_train_ffnn_4)
rnn_scaled_train_ffnn_5 = X_scaler_ffnn_5.fit_transform(rnn_train_ffnn_5)

Y_train_Scaled = Y_scaler.fit_transform(Y_train)
Y_test_Scaled = Y_scaler.transform(Y_test)

X_train_Scaled_LSTM = np.hstack(
    (rnn_scaled_train_LSTM_1, rnn_scaled_train_LSTM_2, rnn_scaled_train_LSTM_3,
     rnn_scaled_train_LSTM_4, rnn_scaled_train_LSTM_5)
).reshape(rnn_train_LSTM_1.shape[0], 5, 48).transpose(0, 2, 1)

X_train_Scaled_ffnn = np.hstack(
    (rnn_scaled_train_ffnn_1, rnn_scaled_train_ffnn_2, rnn_scaled_train_ffnn_3,
     rnn_scaled_train_ffnn_4, rnn_scaled_train_ffnn_5)
).reshape(rnn_train_ffnn_1.shape[0], 5, 16).transpose(0, 2, 1)

X_test_Scaled_LSTM = np.hstack(
    (X_scaler_LSTM_1.transform(rnn_test_LSTM_1), X_scaler_LSTM_2.transform(rnn_test_LSTM_2),
     X_scaler_LSTM_3.transform(rnn_test_LSTM_3), X_scaler_LSTM_4.transform(rnn_test_LSTM_4),
     X_scaler_LSTM_5.transform(rnn_test_LSTM_5))
).reshape(rnn_test_LSTM_1.shape[0], 5, 48).transpose(0, 2, 1)

X_test_Scaled_ffnn = np.hstack(
    (X_scaler_ffnn_1.transform(rnn_test_ffnn_1), X_scaler_ffnn_2.transform(rnn_test_ffnn_2),
     X_scaler_ffnn_3.transform(rnn_test_ffnn_3), X_scaler_ffnn_4.transform(rnn_test_ffnn_4),
     X_scaler_ffnn_5.transform(rnn_test_ffnn_5))
).reshape(rnn_test_ffnn_1.shape[0], 5, 16).transpose(0, 2, 1)

pp ={
    'lstm_layer':[1,2,3],
    'dnn_layer':[1,2,3],
    'n_layer_o':[1,2,3],
    
    'lstm_neurons': [16, 32, 64, 128, 192, 256],
    'dnn_neurons':  [16, 32, 64, 128, 192, 256],
    'n_neuro_m':    [16, 32, 64, 128, 192, 256],
    'n_neuro_o':    [16, 32, 64, 128, 192, 256],

    'lstm_activation':['relu', 'sigmoid',  'tanh'],
    'dnn_activation':['relu', 'sigmoid',  'tanh'],
    'activation_m':['relu', 'sigmoid',  'tanh'],
    'activation_o':['relu', 'sigmoid',  'tanh'],

    'dropout_rate_lstm': list(np.linspace(0.0,0.4, 10)),
    'dropout_rate_dnn': list(np.linspace(0.0,0.4, 10)),   
    
    'learning_rate': list(np.linspace(0.0001,0.02, 10)),        
    'batch_size': [4, 8, 16, 32],
    'epochs': [300]
}
i_shape_lstm = (X_train_Scaled_LSTM.shape[1], X_train_Scaled_LSTM.shape[2])
i_shape_ffnn = (X_train_Scaled_ffnn.shape[1], X_train_Scaled_ffnn.shape[2])


def create_model(x_train, y_train, x_val, y_val, params):
    visible1 = Input(shape=(i_shape_lstm))
    x1=visible1
    
    for i in range(params['lstm_layer']):
        x1= LSTM(params['lstm_neurons'], return_sequences= True, activation=params['lstm_activation'], 
                input_shape=i_shape_lstm)(x1)
    x1 = Dropout(params['dropout_rate_lstm'])(x1)
    
    input_1 = Flatten()(x1)   
    
    
    visible2 = Input(shape=(i_shape_ffnn))   
    x2= visible2 
    
    for i in range(params['dnn_layer']):
        x2 = Dense(params['dnn_neurons'], params['dnn_activation'])(x2)
    x2 = Dropout(params['dropout_rate_dnn'])(x2)
    
    input_2 = Flatten()(x2) 
            
    
    merged = concatenate([input_1, input_2])
    
    a = Dense(params['n_neuro_m'], params['activation_m'])(merged)
    b = Dense(params['n_neuro_m'], params['activation_m'])(merged)
    c = Dense(params['n_neuro_m'], params['activation_m'])(merged)
    d = Dense(params['n_neuro_m'], params['activation_m'])(merged)
    e = Dense(params['n_neuro_m'], params['activation_m'])(merged)
        
            
    for i in range(params['n_layer_o']):
            a = Dense(params['n_neuro_o'], params['activation_o'])(a) 
    output_1 =  Dense(16, name='out_10')(a)
    
    for i in range(params['n_layer_o']):
            b = Dense(params['n_neuro_o'], params['activation_o'])(b) 
    output_2 =  Dense(16, name='out_30')(b)
    
    for i in range(params['n_layer_o']):
            c = Dense(params['n_neuro_o'], params['activation_o'])(c) 
    output_3 =  Dense(16, name='out_50')(c)
    
    for i in range(params['n_layer_o']):
            d = Dense(params['n_neuro_o'], params['activation_o'])(d) 
    output_4 =  Dense(16, name='out_70')(d)
    
    for i in range(params['n_layer_o']):
            e = Dense(params['n_neuro_o'], params['activation_o'])(e) 
    output_5 =  Dense(16, name='out_90')(e)
    
    
    model = Model(inputs=[visible1, visible2], outputs=[output_1, output_2, output_3, output_4, output_5])    
    opt = Adam(learning_rate= params['learning_rate'])

    model.compile(loss=[loss_10th_p, loss_30th_p, loss_50th_p, loss_70th_p, loss_90th_p], optimizer=opt)


    out = model.fit(x=x_train, y=y_train, validation_data=[x_val, y_val],
                    batch_size=params['batch_size'], epochs=params['epochs'], verbose=2,
                    callbacks=[talos.utils.early_stopper(epochs=params['epochs'], mode='moderate',
                                                         min_delta=0.001, monitor='val_loss')])
    return out, model


h = talos.Scan(x=[X_train_Scaled_LSTM, X_train_Scaled_ffnn], y=[Y_train_Scaled,Y_train_Scaled,Y_train_Scaled,Y_train_Scaled,Y_train_Scaled],
               x_val=[X_test_Scaled_LSTM, X_test_Scaled_ffnn], y_val=[Y_test_Scaled, Y_test_Scaled, Y_test_Scaled, Y_test_Scaled, Y_test_Scaled],
               params=pp, model=create_model, experiment_name='MH_DNN_Q', round_limit=30, print_params=True)

h.data.sort_values(by='val_loss', ascending=True)



def create_model():
    visible1 = Input(shape=(i_shape_lstm))
    x1=visible1
    
    for i in range(1):
        x1= LSTM(64, return_sequences= True, activation='sigmoid', input_shape=i_shape_lstm)(x1)
    x1 = Dropout(0.222222)(x1)
    
    input_1 = Flatten()(x1)   
    
    
    visible2 = Input(shape=(i_shape_ffnn))   
    x2= visible2 
    
    for i in range(2):
        x2 = Dense(64, 'tanh')(x2)
    x2 = Dropout(0.088889)(x2)
    
    input_2 = Flatten()(x2) 
            
    
    merged = concatenate([input_1, input_2])
    
    a = Dense(128, 'tanh')(merged)
    b = Dense(128, 'tanh')(merged)
    c = Dense(128, 'tanh')(merged)
    d = Dense(128, 'tanh')(merged)
    e = Dense(128, 'tanh')(merged)
        
            
    for i in range(1):
            a = Dense(128, 'relu')(a) 
    output_1 =  Dense(16, name='out_10')(a)
    
    for i in range(1):
            b = Dense(128, 'relu')(b) 
    output_2 =  Dense(16, name='out_30')(b)
    
    for i in range(1):
            c = Dense(128, 'relu')(c) 
    output_3 =  Dense(16, name='out_50')(c)
    
    for i in range(1):
            d = Dense(128, 'relu')(d) 
    output_4 =  Dense(16, name='out_70')(d)
    
    for i in range(1):
            e = Dense(128, 'relu')(e) 
    output_5 =  Dense(16, name='out_90')(e)
    
    
    model = Model(inputs=[visible1, visible2], outputs=[output_1, output_2, output_3, output_4, output_5])    
    opt = Adam(learning_rate= 0.000100)

    model.compile(loss=[loss_10th_p, loss_30th_p, loss_50th_p, loss_70th_p, loss_90th_p], optimizer=opt)
    return model

mmo = KerasRegressor(build_fn=create_model, epochs=200, batch_size=32, verbose=2)


In [ ]:
#BM Talos Multi-headed(MH) network year 1 hyperparameter search w/results
#full hyperparameter seaarch talos DNN/Multi-headed(MH) model

#months 1-10 search

import numpy as np
import pandas as pd
from pandas import concat
import matplotlib.pyplot as plt
from functools import reduce
import importlib
import datetime as dt
from datetime import datetime
from math import floor
from math import sqrt

# tensorflow &keras
import tensorflow as tf
from keras.layers import Input
from keras.models import Model
from tensorflow.keras.layers import GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras import optimizers
from tensorflow.keras import initializers
from tensorflow.keras.layers import concatenate

from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
# from keras.layers import concatenate
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import LeakyReLU

LeakyReLU = LeakyReLU(alpha=0.1)
from keras.activations import *

from keras.layers import LeakyReLU

LeakyReLU = LeakyReLU(alpha=0.1)

# sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn import metrics
from sklearn import preprocessing as prep
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, mean_absolute_error
from sklearn.metrics import mean_squared_error as MSE
import talos
import tensorflow.keras.backend as K

import warnings

warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
pd.set_option("display.max_columns", None)

date_format = "%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("C:/Users/coconnor/Documents/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_parser=date_parse)
# dat = pd.read_csv("./data/BM_data.csv", index_col="SettlementPeriod", parse_dates=True,
#                   date_parser=date_parse)
dat = pd.read_csv("/home/ciaran/Documents/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_parser=date_parse)


dat = dat.drop(["index"], axis=1)
dat = pd.DataFrame(dat)
dat = dat.bfill(axis='rows')
dat = dat.ffill(axis='rows')
dat = dat._get_numeric_data()

Y=dat.iloc[:, 0:16]
X=dat.iloc[:,16:]
X_train=X.iloc[:11666,:]
Y_train=Y.iloc[:11666,:]
X_test=X.iloc[11666:13155,:]
Y_test=Y.iloc[11666:13155,:]

def qloss(qs, y_true, y_pred):
    # Pinball loss for multiple quantiles
    q = tf.constant(np.array([qs]), dtype=tf.float32)
    e = y_true - y_pred
    v = tf.maximum(q*e, (q-1)*e)
    return K.mean(v)

loss_10th_p = lambda y_true, y_pred: qloss(0.1, y_true, y_pred)
loss_30th_p = lambda y_true, y_pred: qloss(0.3, y_true, y_pred)
loss_50th_p = lambda y_true, y_pred: qloss(0.5, y_true, y_pred)
loss_70th_p = lambda y_true, y_pred: qloss(0.7, y_true, y_pred)
loss_90th_p = lambda y_true, y_pred: qloss(0.9, y_true, y_pred)

rnn_train_LSTM_1 = X_train.loc[:,"lag_-3x1":"lag_-50x1"]
rnn_train_LSTM_2 = X_train.loc[:,"lag_-3x2": "lag_-50x2"]
rnn_train_LSTM_3 = X_train.loc[:,"lag_-2x3":"lag_-49x3"]
rnn_train_LSTM_4 = X_train.loc[:,"lag_0x6":"lag_-47x6"]
rnn_train_LSTM_5 = X_train.loc[:,"lag_-2x12": "lag_-49x12"]

rnn_test_LSTM_1 = X_test.loc[:,"lag_-3x1": "lag_-50x1"]
rnn_test_LSTM_2 = X_test.loc[:,"lag_-3x2": "lag_-50x2"]
rnn_test_LSTM_3 = X_test.loc[:,"lag_-2x3": "lag_-49x3"]
rnn_test_LSTM_4 = X_test.loc[:,"lag_0x6" : "lag_-47x6"]
rnn_test_LSTM_5 = X_test.loc[:,"lag_-2x12": "lag_-49x12"]

rnn_train_ffnn_1 = X_train.loc[:,"lag_2x7": "lag_17x7"]
rnn_train_ffnn_2 = X_train.loc[:,"lag_2x8": "lag_17x8"]
rnn_train_ffnn_3 = X_train.loc[:,"lag_2x9": "lag_17x9"]
rnn_train_ffnn_4 = X_train.loc[:,"lag_2x10": "lag_17x10"]
rnn_train_ffnn_5 = X_train.loc[:,"lag_2x11": "lag_17x11"]

rnn_test_ffnn_1 = X_test.loc[:,"lag_2x7":"lag_17x7"]
rnn_test_ffnn_2 = X_test.loc[:,"lag_2x8": "lag_17x8"]
rnn_test_ffnn_3 = X_test.loc[:,"lag_2x9": "lag_17x9"]
rnn_test_ffnn_4 = X_test.loc[:,"lag_2x10":"lag_17x10"]
rnn_test_ffnn_5 = X_test.loc[:,"lag_2x11": "lag_17x11"]

rnn_Y = Y_train.loc[:,"lag_2y": "lag_17y"]


X_scaler_LSTM_1 = preprocessing.MinMaxScaler()
X_scaler_LSTM_2 = preprocessing.MinMaxScaler()
X_scaler_LSTM_3 = preprocessing.MinMaxScaler()
X_scaler_LSTM_4 = preprocessing.MinMaxScaler()
X_scaler_LSTM_5 = preprocessing.MinMaxScaler()

X_scaler_ffnn_1 = preprocessing.MinMaxScaler()
X_scaler_ffnn_2 = preprocessing.MinMaxScaler()
X_scaler_ffnn_3 = preprocessing.MinMaxScaler()
X_scaler_ffnn_4 = preprocessing.MinMaxScaler()
X_scaler_ffnn_5 = preprocessing.MinMaxScaler()

Y_scaler = preprocessing.MinMaxScaler()

rnn_scaled_train_LSTM_1 = X_scaler_LSTM_1.fit_transform(rnn_train_LSTM_1)
rnn_scaled_train_LSTM_2 = X_scaler_LSTM_2.fit_transform(rnn_train_LSTM_2)
rnn_scaled_train_LSTM_3 = X_scaler_LSTM_3.fit_transform(rnn_train_LSTM_3)
rnn_scaled_train_LSTM_4 = X_scaler_LSTM_4.fit_transform(rnn_train_LSTM_4)
rnn_scaled_train_LSTM_5 = X_scaler_LSTM_5.fit_transform(rnn_train_LSTM_5)

rnn_scaled_train_ffnn_1 = X_scaler_ffnn_1.fit_transform(rnn_train_ffnn_1)
rnn_scaled_train_ffnn_2 = X_scaler_ffnn_2.fit_transform(rnn_train_ffnn_2)
rnn_scaled_train_ffnn_3 = X_scaler_ffnn_3.fit_transform(rnn_train_ffnn_3)
rnn_scaled_train_ffnn_4 = X_scaler_ffnn_4.fit_transform(rnn_train_ffnn_4)
rnn_scaled_train_ffnn_5 = X_scaler_ffnn_5.fit_transform(rnn_train_ffnn_5)

Y_train_Scaled = Y_scaler.fit_transform(Y_train)
Y_test_Scaled = Y_scaler.transform(Y_test)

X_train_Scaled_LSTM = np.hstack(
    (rnn_scaled_train_LSTM_1, rnn_scaled_train_LSTM_2, rnn_scaled_train_LSTM_3,
     rnn_scaled_train_LSTM_4, rnn_scaled_train_LSTM_5)
).reshape(rnn_train_LSTM_1.shape[0], 5, 48).transpose(0, 2, 1)

X_train_Scaled_ffnn = np.hstack(
    (rnn_scaled_train_ffnn_1, rnn_scaled_train_ffnn_2, rnn_scaled_train_ffnn_3,
     rnn_scaled_train_ffnn_4, rnn_scaled_train_ffnn_5)
).reshape(rnn_train_ffnn_1.shape[0], 5, 16).transpose(0, 2, 1)

X_test_Scaled_LSTM = np.hstack(
    (X_scaler_LSTM_1.transform(rnn_test_LSTM_1), X_scaler_LSTM_2.transform(rnn_test_LSTM_2),
     X_scaler_LSTM_3.transform(rnn_test_LSTM_3), X_scaler_LSTM_4.transform(rnn_test_LSTM_4),
     X_scaler_LSTM_5.transform(rnn_test_LSTM_5))
).reshape(rnn_test_LSTM_1.shape[0], 5, 48).transpose(0, 2, 1)

X_test_Scaled_ffnn = np.hstack(
    (X_scaler_ffnn_1.transform(rnn_test_ffnn_1), X_scaler_ffnn_2.transform(rnn_test_ffnn_2),
     X_scaler_ffnn_3.transform(rnn_test_ffnn_3), X_scaler_ffnn_4.transform(rnn_test_ffnn_4),
     X_scaler_ffnn_5.transform(rnn_test_ffnn_5))
).reshape(rnn_test_ffnn_1.shape[0], 5, 16).transpose(0, 2, 1)

pp ={
    'lstm_layer':[1,2,3],
    'dnn_layer':[1,2,3],
    'n_layer_o':[1,2,3],
    
    'lstm_neurons': [16, 32, 64, 128, 192, 256],
    'dnn_neurons':  [16, 32, 64, 128, 192, 256],
    'n_neuro_m':    [16, 32, 64, 128, 192, 256],
    'n_neuro_o':    [16, 32, 64, 128, 192, 256],

    'lstm_activation':['relu', 'sigmoid',  'tanh'],
    'dnn_activation':['relu', 'sigmoid',  'tanh'],
    'activation_m':['relu', 'sigmoid',  'tanh'],
    'activation_o':['relu', 'sigmoid',  'tanh'],

    'dropout_rate_lstm': list(np.linspace(0.0,0.4, 10)),
    'dropout_rate_dnn': list(np.linspace(0.0,0.4, 10)),   
    
    'learning_rate': list(np.linspace(0.0001,0.02, 10)),        
    'batch_size': [4, 8, 16, 32],
    'epochs': [500]
}
i_shape_lstm = (X_train_Scaled_LSTM.shape[1], X_train_Scaled_LSTM.shape[2])
i_shape_ffnn = (X_train_Scaled_ffnn.shape[1], X_train_Scaled_ffnn.shape[2])


def create_model(x_train, y_train, x_val, y_val, params):
    visible1 = Input(shape=(i_shape_lstm))
    x1=visible1
    
    for i in range(params['lstm_layer']):
        x1= LSTM(params['lstm_neurons'], return_sequences= True, activation=params['lstm_activation'], 
                input_shape=i_shape_lstm)(x1)
    x1 = Dropout(params['dropout_rate_lstm'])(x1)
    
    input_1 = Flatten()(x1)   
    
    
    visible2 = Input(shape=(i_shape_ffnn))   
    x2= visible2 
    
    for i in range(params['dnn_layer']):
        x2 = Dense(params['dnn_neurons'], params['dnn_activation'])(x2)
    x2 = Dropout(params['dropout_rate_dnn'])(x2)
    
    input_2 = Flatten()(x2) 
            
    
    merged = concatenate([input_1, input_2])
    
    a = Dense(params['n_neuro_m'], params['activation_m'])(merged)
    b = Dense(params['n_neuro_m'], params['activation_m'])(merged)
    c = Dense(params['n_neuro_m'], params['activation_m'])(merged)
    d = Dense(params['n_neuro_m'], params['activation_m'])(merged)
    e = Dense(params['n_neuro_m'], params['activation_m'])(merged)
        
            
    for i in range(params['n_layer_o']):
            a = Dense(params['n_neuro_o'], params['activation_o'])(a) 
    output_1 =  Dense(16, name='out_10')(a)
    
    for i in range(params['n_layer_o']):
            b = Dense(params['n_neuro_o'], params['activation_o'])(b) 
    output_2 =  Dense(16, name='out_30')(b)
    
    for i in range(params['n_layer_o']):
            c = Dense(params['n_neuro_o'], params['activation_o'])(c) 
    output_3 =  Dense(16, name='out_50')(c)
    
    for i in range(params['n_layer_o']):
            d = Dense(params['n_neuro_o'], params['activation_o'])(d) 
    output_4 =  Dense(16, name='out_70')(d)
    
    for i in range(params['n_layer_o']):
            e = Dense(params['n_neuro_o'], params['activation_o'])(e) 
    output_5 =  Dense(16, name='out_90')(e)
    
    
    model = Model(inputs=[visible1, visible2], outputs=[output_1, output_2, output_3, output_4, output_5])    
    opt = Adam(learning_rate= params['learning_rate'])

    model.compile(loss=[loss_10th_p, loss_30th_p, loss_50th_p, loss_70th_p, loss_90th_p], optimizer=opt)


    out = model.fit(x=x_train, y=y_train, validation_data=[x_val, y_val],
                    batch_size=params['batch_size'], epochs=params['epochs'], verbose=2,
                    callbacks=[talos.utils.early_stopper(epochs=params['epochs'], mode='moderate',
                                                         min_delta=0.001, monitor='val_loss')])
    return out, model


h = talos.Scan(x=[X_train_Scaled_LSTM, X_train_Scaled_ffnn], y=[Y_train_Scaled,Y_train_Scaled,Y_train_Scaled,Y_train_Scaled,Y_train_Scaled],
               x_val=[X_test_Scaled_LSTM, X_test_Scaled_ffnn], y_val=[Y_test_Scaled, Y_test_Scaled, Y_test_Scaled, Y_test_Scaled, Y_test_Scaled],
               params=pp, model=create_model, experiment_name='MH_DNN_Q', round_limit=30, print_params=True)

h.data.sort_values(by='val_loss', ascending=True)



def create_model():
    visible1 = Input(shape=(i_shape_lstm))
    x1=visible1
    
    for i in range(3):
        x1= LSTM(32, return_sequences= True, activation='sigmoid', input_shape=i_shape_lstm)(x1)
    x1 = Dropout(0.311111)(x1)
    
    input_1 = Flatten()(x1)   
    
    
    visible2 = Input(shape=(i_shape_ffnn))   
    x2= visible2 
    
    for i in range(2):
        x2 = Dense(32, 'sigmoid')(x2)
    x2 = Dropout(0.400000)(x2)
    
    input_2 = Flatten()(x2) 
            
    
    merged = concatenate([input_1, input_2])
    
    a = Dense(128, 'sigmoid')(merged)
    b = Dense(128, 'sigmoid')(merged)
    c = Dense(128, 'sigmoid')(merged)
    d = Dense(128, 'sigmoid')(merged)
    e = Dense(128, 'sigmoid')(merged)
        
            
    for i in range(3):
            a = Dense(16, 'tanh')(a) 
    output_1 =  Dense(16, name='out_10')(a)
    
    for i in range(3):
            b = Dense(16, 'tanh')(b) 
    output_2 =  Dense(16, name='out_30')(b)
    
    for i in range(3):
            c = Dense(16, 'tanh')(c) 
    output_3 =  Dense(16, name='out_50')(c)
    
    for i in range(3):
            d = Dense(16, 'tanh')(d) 
    output_4 =  Dense(16, name='out_70')(d)
    
    for i in range(3):
            e = Dense(16, 'tanh')(e) 
    output_5 =  Dense(16, name='out_90')(e)
    
    
    model = Model(inputs=[visible1, visible2], outputs=[output_1, output_2, output_3, output_4, output_5])    
    opt = Adam(learning_rate= 0.000100)

    model.compile(loss=[loss_10th_p, loss_30th_p, loss_50th_p, loss_70th_p, loss_90th_p], optimizer=opt)
    return model


mmo = KerasRegressor(build_fn=create_model, epochs=200, batch_size=4, verbose=2)


In [ ]:
#BM Talos Multi-headed(MH) network year 1 hyperparameter search w/results
#full hyperparameter seaarch talos DNN/Multi-headed(MH) model

#months 1-13 search
import numpy as np
import pandas as pd
from pandas import concat
import matplotlib.pyplot as plt
from functools import reduce
import importlib
import datetime as dt
from datetime import datetime
from math import floor
from math import sqrt

# tensorflow &keras
import tensorflow as tf
from keras.layers import Input
from keras.models import Model
from tensorflow.keras.layers import GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras import optimizers
from tensorflow.keras import initializers
from tensorflow.keras.layers import concatenate

from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
# from keras.layers import concatenate
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import LeakyReLU

LeakyReLU = LeakyReLU(alpha=0.1)
from keras.activations import *

from keras.layers import LeakyReLU

LeakyReLU = LeakyReLU(alpha=0.1)

# sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn import metrics
from sklearn import preprocessing as prep
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, mean_absolute_error
from sklearn.metrics import mean_squared_error as MSE
import talos
import tensorflow.keras.backend as K

import warnings

warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
pd.set_option("display.max_columns", None)

date_format = "%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("C:/Users/coconnor/Documents/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_parser=date_parse)
# dat = pd.read_csv("./data/BM_data.csv", index_col="SettlementPeriod", parse_dates=True,
#                   date_parser=date_parse)
dat = pd.read_csv("/home/ciaran/Documents/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_parser=date_parse)
# dat = pd.read_csv("/home/coconnor/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_parser=date_parse)


dat = dat.drop(["index"], axis=1)
dat = pd.DataFrame(dat)
dat = dat.bfill(axis='rows')
dat = dat.ffill(axis='rows')
dat = dat._get_numeric_data()

Y=dat.iloc[:, 0:16]
X=dat.iloc[:,16:]
X_train=X.iloc[:16082,:]
Y_train=Y.iloc[:16082,:]
X_test=X.iloc[16082:17523,:]
Y_test=Y.iloc[16082:17523,:]

def qloss(qs, y_true, y_pred):
    # Pinball loss for multiple quantiles
    q = tf.constant(np.array([qs]), dtype=tf.float32)
    e = y_true - y_pred
    v = tf.maximum(q*e, (q-1)*e)
    return K.mean(v)

loss_10th_p = lambda y_true, y_pred: qloss(0.1, y_true, y_pred)
loss_30th_p = lambda y_true, y_pred: qloss(0.3, y_true, y_pred)
loss_50th_p = lambda y_true, y_pred: qloss(0.5, y_true, y_pred)
loss_70th_p = lambda y_true, y_pred: qloss(0.7, y_true, y_pred)
loss_90th_p = lambda y_true, y_pred: qloss(0.9, y_true, y_pred)

rnn_train_LSTM_1 = X_train.loc[:,"lag_-3x1":"lag_-50x1"]
rnn_train_LSTM_2 = X_train.loc[:,"lag_-3x2": "lag_-50x2"]
rnn_train_LSTM_3 = X_train.loc[:,"lag_-2x3":"lag_-49x3"]
rnn_train_LSTM_4 = X_train.loc[:,"lag_0x6":"lag_-47x6"]
rnn_train_LSTM_5 = X_train.loc[:,"lag_-2x12": "lag_-49x12"]

rnn_test_LSTM_1 = X_test.loc[:,"lag_-3x1": "lag_-50x1"]
rnn_test_LSTM_2 = X_test.loc[:,"lag_-3x2": "lag_-50x2"]
rnn_test_LSTM_3 = X_test.loc[:,"lag_-2x3": "lag_-49x3"]
rnn_test_LSTM_4 = X_test.loc[:,"lag_0x6" : "lag_-47x6"]
rnn_test_LSTM_5 = X_test.loc[:,"lag_-2x12": "lag_-49x12"]

rnn_train_ffnn_1 = X_train.loc[:,"lag_2x7": "lag_17x7"]
rnn_train_ffnn_2 = X_train.loc[:,"lag_2x8": "lag_17x8"]
rnn_train_ffnn_3 = X_train.loc[:,"lag_2x9": "lag_17x9"]
rnn_train_ffnn_4 = X_train.loc[:,"lag_2x10": "lag_17x10"]
rnn_train_ffnn_5 = X_train.loc[:,"lag_2x11": "lag_17x11"]

rnn_test_ffnn_1 = X_test.loc[:,"lag_2x7":"lag_17x7"]
rnn_test_ffnn_2 = X_test.loc[:,"lag_2x8": "lag_17x8"]
rnn_test_ffnn_3 = X_test.loc[:,"lag_2x9": "lag_17x9"]
rnn_test_ffnn_4 = X_test.loc[:,"lag_2x10":"lag_17x10"]
rnn_test_ffnn_5 = X_test.loc[:,"lag_2x11": "lag_17x11"]

rnn_Y = Y_train.loc[:,"lag_2y": "lag_17y"]


X_scaler_LSTM_1 = preprocessing.MinMaxScaler()
X_scaler_LSTM_2 = preprocessing.MinMaxScaler()
X_scaler_LSTM_3 = preprocessing.MinMaxScaler()
X_scaler_LSTM_4 = preprocessing.MinMaxScaler()
X_scaler_LSTM_5 = preprocessing.MinMaxScaler()

X_scaler_ffnn_1 = preprocessing.MinMaxScaler()
X_scaler_ffnn_2 = preprocessing.MinMaxScaler()
X_scaler_ffnn_3 = preprocessing.MinMaxScaler()
X_scaler_ffnn_4 = preprocessing.MinMaxScaler()
X_scaler_ffnn_5 = preprocessing.MinMaxScaler()

Y_scaler = preprocessing.MinMaxScaler()

rnn_scaled_train_LSTM_1 = X_scaler_LSTM_1.fit_transform(rnn_train_LSTM_1)
rnn_scaled_train_LSTM_2 = X_scaler_LSTM_2.fit_transform(rnn_train_LSTM_2)
rnn_scaled_train_LSTM_3 = X_scaler_LSTM_3.fit_transform(rnn_train_LSTM_3)
rnn_scaled_train_LSTM_4 = X_scaler_LSTM_4.fit_transform(rnn_train_LSTM_4)
rnn_scaled_train_LSTM_5 = X_scaler_LSTM_5.fit_transform(rnn_train_LSTM_5)

rnn_scaled_train_ffnn_1 = X_scaler_ffnn_1.fit_transform(rnn_train_ffnn_1)
rnn_scaled_train_ffnn_2 = X_scaler_ffnn_2.fit_transform(rnn_train_ffnn_2)
rnn_scaled_train_ffnn_3 = X_scaler_ffnn_3.fit_transform(rnn_train_ffnn_3)
rnn_scaled_train_ffnn_4 = X_scaler_ffnn_4.fit_transform(rnn_train_ffnn_4)
rnn_scaled_train_ffnn_5 = X_scaler_ffnn_5.fit_transform(rnn_train_ffnn_5)

Y_train_Scaled = Y_scaler.fit_transform(Y_train)
Y_test_Scaled = Y_scaler.transform(Y_test)

X_train_Scaled_LSTM = np.hstack(
    (rnn_scaled_train_LSTM_1, rnn_scaled_train_LSTM_2, rnn_scaled_train_LSTM_3,
     rnn_scaled_train_LSTM_4, rnn_scaled_train_LSTM_5)
).reshape(rnn_train_LSTM_1.shape[0], 5, 48).transpose(0, 2, 1)

X_train_Scaled_ffnn = np.hstack(
    (rnn_scaled_train_ffnn_1, rnn_scaled_train_ffnn_2, rnn_scaled_train_ffnn_3,
     rnn_scaled_train_ffnn_4, rnn_scaled_train_ffnn_5)
).reshape(rnn_train_ffnn_1.shape[0], 5, 16).transpose(0, 2, 1)

X_test_Scaled_LSTM = np.hstack(
    (X_scaler_LSTM_1.transform(rnn_test_LSTM_1), X_scaler_LSTM_2.transform(rnn_test_LSTM_2),
     X_scaler_LSTM_3.transform(rnn_test_LSTM_3), X_scaler_LSTM_4.transform(rnn_test_LSTM_4),
     X_scaler_LSTM_5.transform(rnn_test_LSTM_5))
).reshape(rnn_test_LSTM_1.shape[0], 5, 48).transpose(0, 2, 1)

X_test_Scaled_ffnn = np.hstack(
    (X_scaler_ffnn_1.transform(rnn_test_ffnn_1), X_scaler_ffnn_2.transform(rnn_test_ffnn_2),
     X_scaler_ffnn_3.transform(rnn_test_ffnn_3), X_scaler_ffnn_4.transform(rnn_test_ffnn_4),
     X_scaler_ffnn_5.transform(rnn_test_ffnn_5))
).reshape(rnn_test_ffnn_1.shape[0], 5, 16).transpose(0, 2, 1)

pp ={

    
    'lstm_neurons': [16, 32, 64, 128, 192, 256],
    'dnn_neurons':  [16, 32, 64, 128, 192, 256],
    'n_neuro_m':    [16, 32, 64, 128, 192, 256],
    'n_neuro_o':    [16, 32, 64, 128, 192, 256],


    'epochs': [300]
}
i_shape_lstm = (X_train_Scaled_LSTM.shape[1], X_train_Scaled_LSTM.shape[2])
i_shape_ffnn = (X_train_Scaled_ffnn.shape[1], X_train_Scaled_ffnn.shape[2])


def create_model(x_train, y_train, x_val, y_val, params):
    visible1 = Input(shape=(i_shape_lstm))
    x1=visible1
    
    for i in range(1):
        x1= LSTM(params['lstm_neurons'], return_sequences= True, activation='sigmoid', 
                input_shape=i_shape_lstm)(x1)
    x1 = Dropout(0.222222)(x1)
    
    input_1 = Flatten()(x1)   
    
    
    visible2 = Input(shape=(i_shape_ffnn))   
    x2= visible2 
    
    for i in range(2):
        x2 = Dense(params['dnn_neurons'], 'tanh')(x2)
    x2 = Dropout(0.088889)(x2)
    
    input_2 = Flatten()(x2) 
            
    
    merged = concatenate([input_1, input_2])
    
    a = Dense(params['n_neuro_m'], 'tanh')(merged)
    b = Dense(params['n_neuro_m'], 'tanh')(merged)
    c = Dense(params['n_neuro_m'], 'tanh')(merged)
    d = Dense(params['n_neuro_m'], 'tanh')(merged)
    e = Dense(params['n_neuro_m'], 'tanh')(merged)
        
            
    for i in range(1):
            a = Dense(params['n_neuro_o'], 'relu')(a) 
    output_1 =  Dense(16, name='out_10')(a)
    
    for i in range(1):
            b = Dense(params['n_neuro_o'], 'relu')(b) 
    output_2 =  Dense(16, name='out_30')(b)
    
    for i in range(1):
            c = Dense(params['n_neuro_o'], 'relu')(c) 
    output_3 =  Dense(16, name='out_50')(c)
    
    for i in range(1):
            d = Dense(params['n_neuro_o'], 'relu')(d) 
    output_4 =  Dense(16, name='out_70')(d)
    
    for i in range(1):
            e = Dense(params['n_neuro_o'], 'relu')(e) 
    output_5 =  Dense(16, name='out_90')(e)
    
    
    model = Model(inputs=[visible1, visible2], outputs=[output_1, output_2, output_3, output_4, output_5])    
    opt = Adam(learning_rate= 0.000100)

    model.compile(loss=[loss_10th_p, loss_30th_p, loss_50th_p, loss_70th_p, loss_90th_p], optimizer=opt)


    out = model.fit(x=x_train, y=y_train, validation_data=[x_val, y_val],
                    batch_size=32, epochs=params['epochs'], verbose=2,
                    callbacks=[talos.utils.early_stopper(epochs=params['epochs'], mode='moderate',
                                                         min_delta=0.001, monitor='val_loss')])
    return out, model


h = talos.Scan(x=[X_train_Scaled_LSTM, X_train_Scaled_ffnn], y=[Y_train_Scaled,Y_train_Scaled,Y_train_Scaled,Y_train_Scaled,Y_train_Scaled],
               x_val=[X_test_Scaled_LSTM, X_test_Scaled_ffnn], y_val=[Y_test_Scaled, Y_test_Scaled, Y_test_Scaled, Y_test_Scaled, Y_test_Scaled],
               params=pp, model=create_model, experiment_name='MH_DNN_Q', round_limit=30, print_params=True)

h.data.sort_values(by='val_loss', ascending=True)



# def create_model():
#     visible1 = Input(shape=(i_shape_lstm))
#     x1=visible1
    
#     for i in range(3):
#         x1= LSTM(16, return_sequences= True, activation='relu', input_shape=i_shape_lstm)(x1)
#     x1 = Dropout(0.222222)(x1)
    
#     input_1 = Flatten()(x1)   
    
    
#     visible2 = Input(shape=(i_shape_ffnn))   
#     x2= visible2 
    
#     for i in range(3):
#         x2 = Dense(32, 'tanh')(x2)
#     x2 = Dropout(0.355556)(x2)
    
#     input_2 = Flatten()(x2) 
            
    
#     merged = concatenate([input_1, input_2])
    
#     a = Dense(32, 'sigmoid')(merged)
#     b = Dense(32, 'sigmoid')(merged)
#     c = Dense(32, 'sigmoid')(merged)
#     d = Dense(32, 'sigmoid')(merged)
#     e = Dense(32, 'sigmoid')(merged)
        
            
#     for i in range(2):
#             a = Dense(128, 'relu')(a) 
#     output_1 =  Dense(16, name='out_10')(a)
    
#     for i in range(2):
#             b = Dense(128, 'relu')(b) 
#     output_2 =  Dense(16, name='out_30')(b)
    
#     for i in range(2):
#             c = Dense(128, 'relu')(c) 
#     output_3 =  Dense(16, name='out_50')(c)
    
#     for i in range(2):
#             d = Dense(128, 'relu')(d) 
#     output_4 =  Dense(16, name='out_70')(d)
    
#     for i in range(2):
#             e = Dense(128, 'relu')(e) 
#     output_5 =  Dense(16, name='out_90')(e)
    
    
#     model = Model(inputs=[visible1, visible2], outputs=[output_1, output_2, output_3, output_4, output_5])    
#     opt = Adam(learning_rate= 0.002311)

#     model.compile(loss=[loss_10th_p, loss_30th_p, loss_50th_p, loss_70th_p, loss_90th_p], optimizer=opt)
#     return model

# mmo = KerasRegressor(build_fn=create_model, epochs=200, batch_size=8, verbose=2)


In [1]:
#BM Talos Multi-Headed network year 1 hyperparameter search w/results
#full hyperparameter seaarch talos DNN/Multi-headed(MH) model

#months 1-16 search
import numpy as np
import pandas as pd
from pandas import concat
import matplotlib.pyplot as plt
from functools import reduce
import importlib
import datetime as dt
from datetime import datetime
from math import floor
from math import sqrt

# tensorflow &keras
import tensorflow as tf
from keras.layers import Input
from keras.models import Model
from tensorflow.keras.layers import GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras import optimizers
from tensorflow.keras import initializers
from tensorflow.keras.layers import concatenate

from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
# from keras.layers import concatenate
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import LeakyReLU

LeakyReLU = LeakyReLU(alpha=0.1)
from keras.activations import *

from keras.layers import LeakyReLU

LeakyReLU = LeakyReLU(alpha=0.1)

# sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn import metrics
from sklearn import preprocessing as prep
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, mean_absolute_error
from sklearn.metrics import mean_squared_error as MSE
import talos
import tensorflow.keras.backend as K

import warnings

warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
pd.set_option("display.max_columns", None)

date_format = "%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("C:/Users/coconnor/Documents/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_parser=date_parse)
# dat = pd.read_csv("./data/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_parser=date_parse)
dat = pd.read_csv("/home/ciaran/Documents/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_parser=date_parse)
# dat = pd.read_csv("/home/coconnor/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_parser=date_parse)


dat = dat.drop(["index"], axis=1)
dat = pd.DataFrame(dat)
dat = dat.bfill(axis='rows')
dat = dat.ffill(axis='rows')
dat = dat._get_numeric_data()

Y=dat.iloc[:, 0:16]
X=dat.iloc[:,16:]
X_train=X.iloc[:20498,:]
Y_train=Y.iloc[:20498,:]
X_test=X.iloc[20498:21843,:]
Y_test=Y.iloc[20498:21843,:]

def qloss(qs, y_true, y_pred):
    # Pinball loss for multiple quantiles
    q = tf.constant(np.array([qs]), dtype=tf.float32)
    e = y_true - y_pred
    v = tf.maximum(q*e, (q-1)*e)
    return K.mean(v)

loss_10th_p = lambda y_true, y_pred: qloss(0.1, y_true, y_pred)
loss_30th_p = lambda y_true, y_pred: qloss(0.3, y_true, y_pred)
loss_50th_p = lambda y_true, y_pred: qloss(0.5, y_true, y_pred)
loss_70th_p = lambda y_true, y_pred: qloss(0.7, y_true, y_pred)
loss_90th_p = lambda y_true, y_pred: qloss(0.9, y_true, y_pred)

rnn_train_LSTM_1 = X_train.loc[:,"lag_-3x1":"lag_-50x1"]
rnn_train_LSTM_2 = X_train.loc[:,"lag_-3x2": "lag_-50x2"]
rnn_train_LSTM_3 = X_train.loc[:,"lag_-2x3":"lag_-49x3"]
rnn_train_LSTM_4 = X_train.loc[:,"lag_0x6":"lag_-47x6"]
rnn_train_LSTM_5 = X_train.loc[:,"lag_-2x12": "lag_-49x12"]

rnn_test_LSTM_1 = X_test.loc[:,"lag_-3x1": "lag_-50x1"]
rnn_test_LSTM_2 = X_test.loc[:,"lag_-3x2": "lag_-50x2"]
rnn_test_LSTM_3 = X_test.loc[:,"lag_-2x3": "lag_-49x3"]
rnn_test_LSTM_4 = X_test.loc[:,"lag_0x6" : "lag_-47x6"]
rnn_test_LSTM_5 = X_test.loc[:,"lag_-2x12": "lag_-49x12"]

rnn_train_ffnn_1 = X_train.loc[:,"lag_2x7": "lag_17x7"]
rnn_train_ffnn_2 = X_train.loc[:,"lag_2x8": "lag_17x8"]
rnn_train_ffnn_3 = X_train.loc[:,"lag_2x9": "lag_17x9"]
rnn_train_ffnn_4 = X_train.loc[:,"lag_2x10": "lag_17x10"]
rnn_train_ffnn_5 = X_train.loc[:,"lag_2x11": "lag_17x11"]

rnn_test_ffnn_1 = X_test.loc[:,"lag_2x7":"lag_17x7"]
rnn_test_ffnn_2 = X_test.loc[:,"lag_2x8": "lag_17x8"]
rnn_test_ffnn_3 = X_test.loc[:,"lag_2x9": "lag_17x9"]
rnn_test_ffnn_4 = X_test.loc[:,"lag_2x10":"lag_17x10"]
rnn_test_ffnn_5 = X_test.loc[:,"lag_2x11": "lag_17x11"]

rnn_Y = Y_train.loc[:,"lag_2y": "lag_17y"]



X_scaler_LSTM_1 = preprocessing.MinMaxScaler()
X_scaler_LSTM_2 = preprocessing.MinMaxScaler()
X_scaler_LSTM_3 = preprocessing.MinMaxScaler()
X_scaler_LSTM_4 = preprocessing.MinMaxScaler()
X_scaler_LSTM_5 = preprocessing.MinMaxScaler()

X_scaler_ffnn_1 = preprocessing.MinMaxScaler()
X_scaler_ffnn_2 = preprocessing.MinMaxScaler()
X_scaler_ffnn_3 = preprocessing.MinMaxScaler()
X_scaler_ffnn_4 = preprocessing.MinMaxScaler()
X_scaler_ffnn_5 = preprocessing.MinMaxScaler()

Y_scaler = preprocessing.MinMaxScaler()

rnn_scaled_train_LSTM_1 = X_scaler_LSTM_1.fit_transform(rnn_train_LSTM_1)
rnn_scaled_train_LSTM_2 = X_scaler_LSTM_2.fit_transform(rnn_train_LSTM_2)
rnn_scaled_train_LSTM_3 = X_scaler_LSTM_3.fit_transform(rnn_train_LSTM_3)
rnn_scaled_train_LSTM_4 = X_scaler_LSTM_4.fit_transform(rnn_train_LSTM_4)
rnn_scaled_train_LSTM_5 = X_scaler_LSTM_5.fit_transform(rnn_train_LSTM_5)

rnn_scaled_train_ffnn_1 = X_scaler_ffnn_1.fit_transform(rnn_train_ffnn_1)
rnn_scaled_train_ffnn_2 = X_scaler_ffnn_2.fit_transform(rnn_train_ffnn_2)
rnn_scaled_train_ffnn_3 = X_scaler_ffnn_3.fit_transform(rnn_train_ffnn_3)
rnn_scaled_train_ffnn_4 = X_scaler_ffnn_4.fit_transform(rnn_train_ffnn_4)
rnn_scaled_train_ffnn_5 = X_scaler_ffnn_5.fit_transform(rnn_train_ffnn_5)

Y_train_Scaled = Y_scaler.fit_transform(Y_train)
Y_test_Scaled = Y_scaler.transform(Y_test)

X_train_Scaled_LSTM = np.hstack(
    (rnn_scaled_train_LSTM_1, rnn_scaled_train_LSTM_2, rnn_scaled_train_LSTM_3,
     rnn_scaled_train_LSTM_4, rnn_scaled_train_LSTM_5)
).reshape(rnn_train_LSTM_1.shape[0], 5, 48).transpose(0, 2, 1)

X_train_Scaled_ffnn = np.hstack(
    (rnn_scaled_train_ffnn_1, rnn_scaled_train_ffnn_2, rnn_scaled_train_ffnn_3,
     rnn_scaled_train_ffnn_4, rnn_scaled_train_ffnn_5)
).reshape(rnn_train_ffnn_1.shape[0], 5, 16).transpose(0, 2, 1)

X_test_Scaled_LSTM = np.hstack(
    (X_scaler_LSTM_1.transform(rnn_test_LSTM_1), X_scaler_LSTM_2.transform(rnn_test_LSTM_2),
     X_scaler_LSTM_3.transform(rnn_test_LSTM_3), X_scaler_LSTM_4.transform(rnn_test_LSTM_4),
     X_scaler_LSTM_5.transform(rnn_test_LSTM_5))
).reshape(rnn_test_LSTM_1.shape[0], 5, 48).transpose(0, 2, 1)

X_test_Scaled_ffnn = np.hstack(
    (X_scaler_ffnn_1.transform(rnn_test_ffnn_1), X_scaler_ffnn_2.transform(rnn_test_ffnn_2),
     X_scaler_ffnn_3.transform(rnn_test_ffnn_3), X_scaler_ffnn_4.transform(rnn_test_ffnn_4),
     X_scaler_ffnn_5.transform(rnn_test_ffnn_5))
).reshape(rnn_test_ffnn_1.shape[0], 5, 16).transpose(0, 2, 1)

pp ={

    
    'lstm_neurons': [16, 32, 64, 128, 192, 256],
    'dnn_neurons':  [16, 32, 64, 128, 192, 256],
    'n_neuro_m':    [16, 32, 64, 128, 192, 256],
    'n_neuro_o':    [16, 32, 64, 128, 192, 256],


    'epochs': [300]
}
i_shape_lstm = (X_train_Scaled_LSTM.shape[1], X_train_Scaled_LSTM.shape[2])
i_shape_ffnn = (X_train_Scaled_ffnn.shape[1], X_train_Scaled_ffnn.shape[2])


def create_model(x_train, y_train, x_val, y_val, params):
    visible1 = Input(shape=(i_shape_lstm))
    x1=visible1
    
    for i in range(1):
        x1= LSTM(params['lstm_neurons'], return_sequences= True, activation='sigmoid', 
                input_shape=i_shape_lstm)(x1)
    x1 = Dropout(0.222222)(x1)
    
    input_1 = Flatten()(x1)   
    
    
    visible2 = Input(shape=(i_shape_ffnn))   
    x2= visible2 
    
    for i in range(2):
        x2 = Dense(params['dnn_neurons'], 'tanh')(x2)
    x2 = Dropout(0.088889)(x2)
    
    input_2 = Flatten()(x2) 
            
    
    merged = concatenate([input_1, input_2])
    
    a = Dense(params['n_neuro_m'], 'tanh')(merged)
    b = Dense(params['n_neuro_m'], 'tanh')(merged)
    c = Dense(params['n_neuro_m'], 'tanh')(merged)
    d = Dense(params['n_neuro_m'], 'tanh')(merged)
    e = Dense(params['n_neuro_m'], 'tanh')(merged)
        
            
    for i in range(1):
            a = Dense(params['n_neuro_o'], 'relu')(a) 
    output_1 =  Dense(16, name='out_10')(a)
    
    for i in range(1):
            b = Dense(params['n_neuro_o'], 'relu')(b) 
    output_2 =  Dense(16, name='out_30')(b)
    
    for i in range(1):
            c = Dense(params['n_neuro_o'], 'relu')(c) 
    output_3 =  Dense(16, name='out_50')(c)
    
    for i in range(1):
            d = Dense(params['n_neuro_o'], 'relu')(d) 
    output_4 =  Dense(16, name='out_70')(d)
    
    for i in range(1):
            e = Dense(params['n_neuro_o'], 'relu')(e) 
    output_5 =  Dense(16, name='out_90')(e)
    
    
    model = Model(inputs=[visible1, visible2], outputs=[output_1, output_2, output_3, output_4, output_5])    
    opt = Adam(learning_rate= 0.000100)

    model.compile(loss=[loss_10th_p, loss_30th_p, loss_50th_p, loss_70th_p, loss_90th_p], optimizer=opt)


    out = model.fit(x=x_train, y=y_train, validation_data=[x_val, y_val],
                    batch_size=32, epochs=params['epochs'], verbose=2,
                    callbacks=[talos.utils.early_stopper(epochs=params['epochs'], mode='moderate',
                                                         min_delta=0.001, monitor='val_loss')])
    return out, model


h = talos.Scan(x=[X_train_Scaled_LSTM, X_train_Scaled_ffnn], y=[Y_train_Scaled,Y_train_Scaled,Y_train_Scaled,Y_train_Scaled,Y_train_Scaled],
               x_val=[X_test_Scaled_LSTM, X_test_Scaled_ffnn], y_val=[Y_test_Scaled, Y_test_Scaled, Y_test_Scaled, Y_test_Scaled, Y_test_Scaled],
               params=pp, model=create_model, experiment_name='MH_DNN_Q', round_limit=30, print_params=True)

h.data.sort_values(by='val_loss', ascending=True)



# def create_model():
#     visible1 = Input(shape=(i_shape_lstm))
#     x1=visible1
    
#     for i in range(3):
#         x1= LSTM(32, return_sequences= True, activation='sigmoid', input_shape=i_shape_lstm)(x1)
#     x1 = Dropout(0.266667)(x1)
    
#     input_1 = Flatten()(x1)   
    
    
#     visible2 = Input(shape=(i_shape_ffnn))   
#     x2= visible2 
    
#     for i in range(1):
#         x2 = Dense(128, 'sigmoid')(x2)
#     x2 = Dropout(0.177778)(x2)
    
#     input_2 = Flatten()(x2) 
            
    
#     merged = concatenate([input_1, input_2])
    
#     a = Dense(256, 'relu')(merged)
#     b = Dense(256, 'relu')(merged)
#     c = Dense(256, 'relu')(merged)
#     d = Dense(256, 'relu')(merged)
#     e = Dense(256, 'relu')(merged)
        
            
#     for i in range(1):
#             a = Dense(128, 'sigmoid')(a) 
#     output_1 =  Dense(16, name='out_10')(a)
    
#     for i in range(1):
#             b = Dense(128, 'sigmoid')(b) 
#     output_2 =  Dense(16, name='out_30')(b)
    
#     for i in range(1):
#             c = Dense(128, 'sigmoid')(c) 
#     output_3 =  Dense(16, name='out_50')(c)
    
#     for i in range(1):
#             d = Dense(128, 'sigmoid')(d) 
#     output_4 =  Dense(16, name='out_70')(d)
    
#     for i in range(1):
#             e = Dense(128, 'sigmoid')(e) 
#     output_5 =  Dense(16, name='out_90')(e)
    
    
#     model = Model(inputs=[visible1, visible2], outputs=[output_1, output_2, output_3, output_4, output_5])    
#     opt = Adam(learning_rate= 0.000100)

#     model.compile(loss=[loss_10th_p, loss_30th_p, loss_50th_p, loss_70th_p, loss_90th_p], optimizer=opt)
#     return model

# mmo = KerasRegressor(build_fn=create_model, epochs=200, batch_size=16, verbose=2)

2022-12-21 15:36:09.276342: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-21 15:36:09.422788: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-21 15:36:10.179131: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-21 15:36:10.179169: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

{'activation_m': 'relu', 'activation_o': 'sigmoid', 'batch_size': 16, 'dnn_activation': 'sigmoid', 'dnn_layer': 1, 'dnn_neurons': 128, 'dropout_rate_dnn': 0.17777777777777778, 'dropout_rate_lstm': 0.26666666666666666, 'epochs': 500, 'learning_rate': 0.0001, 'lstm_activation': 'sigmoid', 'lstm_layer': 3, 'lstm_neurons': 32, 'n_layer_o': 1, 'n_neuro_m': 256, 'n_neuro_o': 128}
Epoch 1/500
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
1282/1282 - 57s - loss: 0.0502 - out_10_loss: 0.0060 - out_30_loss: 0.0110 - out_50_loss: 0.0127 - out_70_loss: 0.0124 - out_90_loss: 0.0082 - val_loss: 0.0401 - val_out_10_loss: 0.0044 - val_out_30_loss: 0.0087 - val_out_50_loss: 0.0106 - val_out_70_loss: 0.0104 - val_out_90_loss: 0.0060 - 57s/epoch - 45ms/step
Epoch 2/500
1282/1282 - 54s - loss: 0.0310 - out_10_loss: 0.0035 - out_30_loss: 0.0065 - out_50_

Epoch 24/500
1282/1282 - 56s - loss: 0.0274 - out_10_loss: 0.0031 - out_30_loss: 0.0057 - out_50_loss: 0.0072 - out_70_loss: 0.0070 - out_90_loss: 0.0044 - val_loss: 0.0348 - val_out_10_loss: 0.0042 - val_out_30_loss: 0.0074 - val_out_50_loss: 0.0092 - val_out_70_loss: 0.0089 - val_out_90_loss: 0.0051 - 56s/epoch - 44ms/step
Epoch 25/500
1282/1282 - 56s - loss: 0.0274 - out_10_loss: 0.0031 - out_30_loss: 0.0057 - out_50_loss: 0.0072 - out_70_loss: 0.0070 - out_90_loss: 0.0044 - val_loss: 0.0351 - val_out_10_loss: 0.0041 - val_out_30_loss: 0.0075 - val_out_50_loss: 0.0091 - val_out_70_loss: 0.0089 - val_out_90_loss: 0.0055 - 56s/epoch - 44ms/step
Epoch 26/500
1282/1282 - 56s - loss: 0.0274 - out_10_loss: 0.0030 - out_30_loss: 0.0057 - out_50_loss: 0.0072 - out_70_loss: 0.0070 - out_90_loss: 0.0044 - val_loss: 0.0356 - val_out_10_loss: 0.0041 - val_out_30_loss: 0.0077 - val_out_50_loss: 0.0096 - val_out_70_loss: 0.0089 - val_out_90_loss: 0.0053 - 56s/epoch - 44ms/step
Epoch 27/500
1282/1

Epoch 50/500
1282/1282 - 56s - loss: 0.0268 - out_10_loss: 0.0030 - out_30_loss: 0.0057 - out_50_loss: 0.0071 - out_70_loss: 0.0068 - out_90_loss: 0.0042 - val_loss: 0.0348 - val_out_10_loss: 0.0040 - val_out_30_loss: 0.0074 - val_out_50_loss: 0.0091 - val_out_70_loss: 0.0088 - val_out_90_loss: 0.0054 - 56s/epoch - 44ms/step
Epoch 51/500
1282/1282 - 56s - loss: 0.0268 - out_10_loss: 0.0030 - out_30_loss: 0.0057 - out_50_loss: 0.0071 - out_70_loss: 0.0068 - out_90_loss: 0.0042 - val_loss: 0.0342 - val_out_10_loss: 0.0040 - val_out_30_loss: 0.0072 - val_out_50_loss: 0.0090 - val_out_70_loss: 0.0086 - val_out_90_loss: 0.0053 - 56s/epoch - 44ms/step
Epoch 52/500
1282/1282 - 56s - loss: 0.0268 - out_10_loss: 0.0030 - out_30_loss: 0.0057 - out_50_loss: 0.0071 - out_70_loss: 0.0068 - out_90_loss: 0.0042 - val_loss: 0.0344 - val_out_10_loss: 0.0040 - val_out_30_loss: 0.0074 - val_out_50_loss: 0.0091 - val_out_70_loss: 0.0087 - val_out_90_loss: 0.0052 - 56s/epoch - 44ms/step
Epoch 53/500
1282/1

Epoch 76/500
1282/1282 - 56s - loss: 0.0264 - out_10_loss: 0.0030 - out_30_loss: 0.0056 - out_50_loss: 0.0070 - out_70_loss: 0.0067 - out_90_loss: 0.0042 - val_loss: 0.0342 - val_out_10_loss: 0.0040 - val_out_30_loss: 0.0072 - val_out_50_loss: 0.0089 - val_out_70_loss: 0.0086 - val_out_90_loss: 0.0054 - 56s/epoch - 44ms/step
Epoch 77/500
1282/1282 - 56s - loss: 0.0264 - out_10_loss: 0.0030 - out_30_loss: 0.0056 - out_50_loss: 0.0070 - out_70_loss: 0.0067 - out_90_loss: 0.0041 - val_loss: 0.0340 - val_out_10_loss: 0.0040 - val_out_30_loss: 0.0072 - val_out_50_loss: 0.0089 - val_out_70_loss: 0.0086 - val_out_90_loss: 0.0053 - 56s/epoch - 44ms/step
Epoch 78/500
1282/1282 - 56s - loss: 0.0265 - out_10_loss: 0.0030 - out_30_loss: 0.0056 - out_50_loss: 0.0070 - out_70_loss: 0.0067 - out_90_loss: 0.0042 - val_loss: 0.0342 - val_out_10_loss: 0.0040 - val_out_30_loss: 0.0072 - val_out_50_loss: 0.0090 - val_out_70_loss: 0.0086 - val_out_90_loss: 0.0052 - 56s/epoch - 44ms/step
Epoch 79/500
1282/1

...

Epoch 25/500
5125/5125 - 90s - loss: 0.6086 - out_10_loss: 0.0904 - out_30_loss: 0.1407 - out_50_loss: 0.1397 - out_70_loss: 0.1462 - out_90_loss: 0.0916 - val_loss: 0.6154 - val_out_10_loss: 0.0872 - val_out_30_loss: 0.1648 - val_out_50_loss: 0.1617 - val_out_70_loss: 0.1146 - val_out_90_loss: 0.0871 - 90s/epoch - 18ms/step
Epoch 26/500
5125/5125 - 90s - loss: 0.6247 - out_10_loss: 0.0893 - out_30_loss: 0.1485 - out_50_loss: 0.1413 - out_70_loss: 0.1515 - out_90_loss: 0.0940 - val_loss: 0.6952 - val_out_10_loss: 0.0766 - val_out_30_loss: 0.1619 - val_out_50_loss: 0.1488 - val_out_70_loss: 0.1909 - val_out_90_loss: 0.1170 - 90s/epoch - 18ms/step
Epoch 27/500
5125/5125 - 91s - loss: 0.6140 - out_10_loss: 0.0887 - out_30_loss: 0.1419 - out_50_loss: 0.1422 - out_70_loss: 0.1492 - out_90_loss: 0.0920 - val_loss: 0.6577 - val_out_10_loss: 0.0586 - val_out_30_loss: 0.1844 - val_out_50_loss: 0.1448 - val_out_70_loss: 0.2033 - val_out_90_loss: 0.0667 - 91s/epoch - 18ms/step
Epoch 28/500
5125/5

Epoch 51/500
5125/5125 - 90s - loss: 0.6221 - out_10_loss: 0.0885 - out_30_loss: 0.1427 - out_50_loss: 0.1438 - out_70_loss: 0.1525 - out_90_loss: 0.0946 - val_loss: 0.6276 - val_out_10_loss: 0.1061 - val_out_30_loss: 0.1564 - val_out_50_loss: 0.1113 - val_out_70_loss: 0.1596 - val_out_90_loss: 0.0942 - 90s/epoch - 18ms/step
Epoch 52/500
5125/5125 - 90s - loss: 0.6199 - out_10_loss: 0.0907 - out_30_loss: 0.1460 - out_50_loss: 0.1450 - out_70_loss: 0.1448 - out_90_loss: 0.0934 - val_loss: 0.5898 - val_out_10_loss: 0.0803 - val_out_30_loss: 0.1941 - val_out_50_loss: 0.0876 - val_out_70_loss: 0.1213 - val_out_90_loss: 0.1064 - 90s/epoch - 18ms/step
Epoch 53/500
5125/5125 - 90s - loss: 0.6250 - out_10_loss: 0.0891 - out_30_loss: 0.1486 - out_50_loss: 0.1460 - out_70_loss: 0.1494 - out_90_loss: 0.0919 - val_loss: 0.6993 - val_out_10_loss: 0.0552 - val_out_30_loss: 0.1672 - val_out_50_loss: 0.1363 - val_out_70_loss: 0.1904 - val_out_90_loss: 0.1503 - 90s/epoch - 18ms/step
Epoch 54/500
5125/5

Epoch 77/500
5125/5125 - 90s - loss: 0.6333 - out_10_loss: 0.0918 - out_30_loss: 0.1429 - out_50_loss: 0.1577 - out_70_loss: 0.1502 - out_90_loss: 0.0907 - val_loss: 0.6040 - val_out_10_loss: 0.0927 - val_out_30_loss: 0.0790 - val_out_50_loss: 0.1783 - val_out_70_loss: 0.1460 - val_out_90_loss: 0.1080 - 90s/epoch - 18ms/step
Epoch 78/500
5125/5125 - 90s - loss: 0.6311 - out_10_loss: 0.0902 - out_30_loss: 0.1450 - out_50_loss: 0.1579 - out_70_loss: 0.1460 - out_90_loss: 0.0921 - val_loss: 0.6926 - val_out_10_loss: 0.1035 - val_out_30_loss: 0.2104 - val_out_50_loss: 0.1617 - val_out_70_loss: 0.1328 - val_out_90_loss: 0.0841 - 90s/epoch - 18ms/step
Epoch 79/500
5125/5125 - 90s - loss: 0.6332 - out_10_loss: 0.0908 - out_30_loss: 0.1437 - out_50_loss: 0.1575 - out_70_loss: 0.1480 - out_90_loss: 0.0933 - val_loss: 0.5918 - val_out_10_loss: 0.1063 - val_out_30_loss: 0.1650 - val_out_50_loss: 0.0785 - val_out_70_loss: 0.1396 - val_out_90_loss: 0.1024 - 90s/epoch - 18ms/step
Epoch 80/500
5125/5

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [127:07:08<00:00, 15254.28s/it]


,start,end,duration,round_epochs,loss,out_10_loss,out_30_loss,out_50_loss,out_70_loss,out_90_loss,val_loss,val_out_10_loss,val_out_30_loss,val_out_50_loss,val_out_70_loss,val_out_90_loss,activation_m,activation_o,batch_size,dnn_activation,dnn_layer,dnn_neurons,dropout_rate_dnn,dropout_rate_lstm,epochs,learning_rate,lstm_activation,lstm_layer,lstm_neurons,n_layer_o,n_neuro_m,n_neuro_o
0,12/21/22-153612,12/21/22-184347,11254.631946,201,0.025000,0.002843,0.005371,0.006612,0.006318,0.003856,0.034025,0.003934,0.007203,0.008906,0.008669,0.005313,relu,sigmoid,16,sigmoid,1,128,0.177778,0.266667,500,0.000100,sigmoid,3,32,1,256,128
24,12/26/22-003359,12/26/22-151948,53149.317812,71,0.026445,0.003030,0.005592,0.006854,0.006699,0.004271,0.034798,0.004002,0.007206,0.009089,0.008981,0.005521,tanh,relu,16,relu,2,64,0.044444,0.088889,500,0.006733,tanh,3,64,2,16,64
16,12/25/22-000259,12/25/22-001828,928.446671,59,0.022758,0.002678,0.005219,0.006088,0.005477,0.003296,0.035965,0.004036,0.007728,0.009385,0.009124,0.005692,sigmoid,sigmoid,32,relu,3,32,0.400000,0.000000,500,0.008944,relu,3,16,1,192,192
8,12/23/22-205914,12/23/22-213927,2413.452098,57,0.017597,0.002045,0.003967,0.004859,0.004298,0.002428,0.036719,0.005151,0.008140,0.009034,0.008774,0.005619,sigmoid,relu,16,tanh,1,128,0.177778,0.400000,500,0.002311,tanh,2,16,2,256,32
14,12/24/22-181217,12/24/22-202203,7785.756500,57,0.017151,0.002066,0.003892,0.004507,0.004175,0.002511,0.037427,0.005264,0.008499,0.009561,0.008657,0.005447,sigmoid,relu,4,relu,1,192,0.311111,0.355556,500,0.000100,relu,3,64,3,64,16
11,12/24/22-021027,12/24/22-153227,48119.998112,53,0.029546,0.003095,0.006342,0.007098,0.007775,0.005236,0.038194,0.004807,0.008338,0.009206,0.009947,0.005896,tanh,relu,8,relu,2,64,0.222222,0.400000,500,0.004522,relu,3,128,3,256,32
25,12/26/22-151949,12/26/22-164216,4947.171242,69,0.031585,0.003629,0.006554,0.008013,0.007993,0.005396,0.038555,0.004875,0.008479,0.009972,0.009594,0.005635,relu,relu,16,relu,3,64,0.044444,0.222222,500,0.017789,sigmoid,2,256,2,32,256
1,12/21/22-184347,12/21/22-195526,4298.856280,62,0.029968,0.003561,0.006464,0.007948,0.007311,0.004685,0.038562,0.004334,0.008513,0.010506,0.009152,0.006057,tanh,relu,8,relu,1,128,0.222222,0.311111,500,0.008944,relu,3,128,1,16,64
28,12/26/22-175034,12/26/22-201845,8891.261251,68,0.030218,0.003380,0.006256,0.007718,0.007688,0.005177,0.038820,0.004316,0.008341,0.010275,0.010083,0.005805,relu,sigmoid,8,tanh,1,64,0.000000,0.088889,500,0.002311,tanh,3,128,2,128,16
3,12/21/22-211619,12/21/22-232721,7862.164874,51,0.030360,0.003401,0.006279,0.007743,0.007715,0.005221,0.038922,0.004349,0.008432,0.010228,0.009734,0.006179,sigmoid,tanh,8,tanh,3,256,0.133333,0.311111,500,0.002311,relu,3,256,1,32,128


In [ ]:
# Y=dat.iloc[:, 0:16]
# X=dat.iloc[:,16:]
# X_train=X.iloc[:7250,:]
# Y_train=Y.iloc[:7250,:]
# X_test=X.iloc[7250:8739,:]
# Y_test=Y.iloc[7250:8739,:]

# Y=dat.iloc[:, 0:16]
# X=dat.iloc[:,16:]
# X_train=X.iloc[:11666,:]
# Y_train=Y.iloc[:11666,:]
# X_test=X.iloc[11666:13155,:]
# Y_test=Y.iloc[11666:13155,:]


# Y=dat.iloc[:, 0:16]
# X=dat.iloc[:,16:]
# X_train=X.iloc[:16082,:]
# Y_train=Y.iloc[:16082,:]
# X_test=X.iloc[16082:17523,:]
# Y_test=Y.iloc[16082:17523,:]

# Y=dat.iloc[:, 0:16]
# X=dat.iloc[:,16:]
# X_train=X.iloc[:20498,:]
# Y_train=Y.iloc[:20498,:]
# X_test=X.iloc[20498:21843,:]
# Y_test=Y.iloc[20498:21843,:]
# Y_train